### standard imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import random
import warnings

from botorch.models.gp_regression import SingleTaskGP
from tqdm import tqdm
from botorch.acquisition.active_learning import qNegIntegratedPosteriorVariance
from botorch.fit import fit_gpytorch_mll
from botorch.utils.transforms import normalize, standardize
from botorch.utils.sampling import draw_sobol_samples
from botorch.exceptions.warnings import InputDataWarning
from gpytorch.mlls import ExactMarginalLogLikelihood
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

warnings.filterwarnings(
    "ignore",
    message="Input data is not standardized.",
    category=InputDataWarning,
)
warnings.filterwarnings("ignore")

### checking my working directory so i can make sure to import my data file correctly


In [3]:
df = pd.read_csv('../datasets/AutoAM_dataset.csv')
df

,Prime Delay,Print Speed,X Offset Correction,Y Offset Correction,Score
0,0.000000,1.000000,0.000000,0.000000,0.339554
1,2.500000,3.000000,0.100000,0.100000,0.000000
2,5.000000,5.000000,-0.100000,-0.100000,0.218576
3,0.000000,0.999931,0.000006,0.000025,0.368919
4,0.000000,5.070133,0.225151,1.000000,0.000000
...,...,...,...,...,...
95,0.742365,1.714610,-0.380317,-0.416497,0.829756
96,0.000000,1.585934,-0.337143,-0.372811,0.922120
97,0.000000,1.568030,-0.335316,-0.365804,0.901296
98,0.000000,1.558826,-0.349834,-0.328645,0.936549


### converting my data into a tensor / and normalizing 

In [4]:
xlower_bounds = []
xupper_bounds = []
x_vals = df.iloc[:,:-1]
for col in x_vals.columns:
    xlower = x_vals[col].min()
    xupper = x_vals[col].max()
    xlower_bounds.append(xlower)
    xupper_bounds.append(xupper)
    
    

In [5]:
xlower_bounds = torch.tensor(xlower_bounds, dtype=torch.double)
xupper_bounds = torch.tensor(xupper_bounds, dtype=torch.double)

xbounds = torch.stack([xlower_bounds, xupper_bounds])
xbounds

#convert numpy array to tensor  

x = torch.tensor(df.iloc[:,:-1].values, dtype=torch.double)

x = torch.tensor(df.iloc[:,:-1].values, dtype=torch.double)

In [6]:
dtype = torch.double
x_vals = df.iloc[:,:-1]
x_tensors = torch.tensor(x_vals.values, dtype=dtype)

print(f'x_tensor.shape',x_tensors.shape)

y_tensors = torch.tensor(df.iloc[:,-1].values).unsqueeze(-1).double()
print(f'y_tensor.shape',y_tensors.shape)

x = normalize(x_tensors, bounds=xbounds)  
y = standardize(y_tensors)




x_tensor.shape torch.Size([100, 4])
y_tensor.shape torch.Size([100, 1])


In [7]:
x_candidates, x_test, y_candidates, y_test = train_test_split(x, y, test_size=0.30, random_state=42)


### we set the bounds here from 0 being the min and 1 being the max

In [8]:
torch.manual_seed(0)
mps_device = torch.device("cpu")
dtype = torch.double



xtest = torch.tensor(x_test, device=mps_device, dtype=dtype)
ytest = torch.tensor(y_test, device=mps_device, dtype=dtype)

xcandidates_original = torch.tensor(x_candidates, device=mps_device, dtype=dtype)
ycandidates_original = torch.tensor(y_candidates, device=mps_device, dtype=dtype)

In [9]:
lower_bound_mcp = []
upper_bound_mcp = []
for i in range(xcandidates_original.shape[1]):
    print(xcandidates_original[:,i].min(), xcandidates_original[:,i].max())
    lower_bound_val = xcandidates_original[:,i].min()
    upper_bound_val = xcandidates_original[:,i].max()
    
    lower_bound_mcp.append(lower_bound_val)
    upper_bound_mcp.append(upper_bound_val)
    
    

tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)


In [10]:
bounds = torch.tensor([lower_bound_mcp, upper_bound_mcp], device='cpu', dtype=dtype)
bounds

tensor([[0., 0., 0., 0.],
        [1., 1., 1., 1.]], dtype=torch.float64)

In [11]:
from botorch.utils.sampling import draw_sobol_samples

mcp = draw_sobol_samples(bounds=bounds, n=1024, q=1, seed=42).squeeze(1)
mcp.shape

torch.Size([1024, 4])

### function for random initial data points by percent

In [12]:
# create function to select random 5% of the data to be used as the initial training
# set and remove it from the candidate set
def random_initial_data(x, y, initial_percent, seed=i):
    np.random.seed(seed)
    n = int(x.shape[0]*initial_percent)
    idx = np.random.choice(x.shape[0], n, replace=False)
    x_initial = x[idx]
    y_initial = y[idx]
    x_candidates = np.delete(x, idx, axis=0)
    y_candidates = np.delete(y, idx, axis=0)
    return x_initial, y_initial, x_candidates, y_candidates


In [ ]:
xcandidates = xcandidates_original.clone()
ycandidates = ycandidates_original.clone()

In [14]:
gp = SingleTaskGP(xcandidates, ycandidates) 
    # gp = SingleTaskGP(xinit, ytrain_,covar_module=rbf_kernel)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)
#predict the y values for the test set
ypred = gp(xtest)
ypred_mean = ypred.mean.detach().numpy()
# pred_y.append(ypred_mean)

#calculate the mean absolute error and the standard deviation for the test set
ymae = mean_absolute_error(ytest, ypred_mean)
ymae

0.28433119799187795

### Make seeds

In [17]:
# set random seeds for 25 runs
random.seed(42)
seeds = [random.randint(1, 5000) for _ in range(25)]
print(seeds)
len(seeds)

[913, 205, 2254, 2007, 1829, 1144, 840, 4468, 713, 4838, 3457, 261, 245, 768, 1792, 1906, 4140, 4932, 218, 4598, 1629, 4465, 3437, 1806, 3680]


25

### QNIPV function

In [ ]:

# def qnipv_runs() -> list:

rand_selection_mae = []
xmax_candidates = []
pred_mae = []
pred_y = []
pred_std = []
qnipv_runs =[]

def find_max_normalized_acqval(tensor_list, qNIVP):
    max_value = None
    max_index = -1
    acq_val_lst = []
    # torch.manual_seed(13)
    for i, tensor_ in enumerate(tensor_list):
        tensor = tensor_.unsqueeze(0)
        qNIVP_val = qNIVP(tensor)
        acq_val_lst.append(qNIVP_val.item())  # Assuming it's a scalar tensor

        # Check if this is the maximum value so far
        if max_value is None or qNIVP_val > max_value:
            max_value = qNIVP_val
            max_index = i

    return max_value, max_index, acq_val_lst

for i in tqdm(seeds):
    xcandidates = xcandidates_original.clone()
    ycandidates = ycandidates_original.clone()
    xinit, yinit, xcandidates, ycandidates = random_initial_data(xcandidates, ycandidates, 0.05, seed=i)
    gp = SingleTaskGP(xinit, yinit)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)
    posterior = gp(xtest)
    ypred = posterior.mean.detach().numpy()
    ystd = posterior.stddev.detach().numpy()
    
    
    # pred_y.append(ypred_mean)
    ymae = mean_absolute_error(ytest, ypred)
    
    pred_mae = []
    pred_y.append(ypred)
    pred_std.append(ystd)
    pred_mae.append(ymae)

    for inner_i in tqdm(range(len(xcandidates))):
        if not len(xcandidates):
            break
        
        qNIVP = qNegIntegratedPosteriorVariance(gp, mc_points= mcp)
        
        
        max_value, max_index, acq_val_lst = find_max_normalized_acqval(xcandidates, qNIVP)
        xmax_candidates.append(max_index)
        
        xinit= torch.cat((xinit, xcandidates[max_index].unsqueeze(0)), 0)
        yinit = torch.cat((yinit, ycandidates[max_index].unsqueeze(0)), 0)
                    
        xcandidates = torch.cat((xcandidates[:max_index], xcandidates[max_index + 1:]))
        ycandidates = torch.cat((ycandidates[:max_index], ycandidates[max_index + 1:]))
        
        gp = SingleTaskGP(xinit, yinit) 
        # gp = SingleTaskGP(xinit, ytrain_,covar_module=rbf_kernel)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_mll(mll)
        #predict the y values for the test set
        ypred = gp(xtest)
        ypred_mean = ypred.mean.detach().numpy()
        pred_y.append(ypred_mean)

        #calculate the mean absolute error and the standard deviation for the test set
        ymae = mean_absolute_error(ytest, ypred_mean)
        # print('mean absolute error: ', ymae)
        pred_mae.append(ymae)
        ystd = gp(xtest).stddev
        ystd = ystd.detach().numpy()
        pred_std.append(ystd)
    qnipv_runs.append(pred_mae)
    
    # return qnipv_runs, gp

100%|██████████| 25/25 [11:20<00:00, 27.22s/it]


### random selection function

In [ ]:
def random_runs() -> list:

    xcandidates_rand = xcandidates_original.clone()
    ycandidates_rand = ycandidates_original.clone()

    rand_xmax_candidates = []
    rand_pred_mae = []
    rand_pred_std = []
    rand_pred_mean = []
    random_mae_seeds =[]

    for i in tqdm(seeds):
       
        xcandidates_rand = xcandidates_original.clone()
        ycandidates_rand = ycandidates_original.clone()
        xinit_rand, yinit_rand, xcandidates_rand, ycandidates_rand = random_initial_data(xcandidates_rand, ycandidates_rand, 0.05, seed=i)
        
        rand_xmax_candidates = []
        rand_pred_mae = []
        rand_pred_std = []
        rand_pred_mean = []

        gp = SingleTaskGP(xinit_rand, yinit_rand) 
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_mll(mll)
        #predict the y values for the test set
        rand_ypred = gp(xtest)
        rand_ypred_mean = rand_ypred.mean.detach().numpy()
        rand_pred_mean.append(rand_ypred_mean)
        #calculate the mean absolute error and the standard deviation for the test set
        rand_ymae = mean_absolute_error(ytest, rand_ypred_mean)
        # print('mean absolute error: ', rand_ymae)
        rand_pred_mae.append(rand_ymae)
            
        rand_ystd = gp(xtest).stddev
        ystd_ = rand_ystd.detach().numpy()
        rand_pred_std.append(ystd_)        
        
        for inner_i in tqdm(range(len(xcandidates_rand))):
            if not len(xcandidates_rand):
                break
            
            rand_select = random.randint(0, len(xcandidates_rand) - 1)
            
            # Add the selected tensor to the training sets
            xinit_rand = torch.cat((xinit_rand, xcandidates_rand[rand_select].unsqueeze(0)), 0)
            yinit_rand = torch.cat((yinit_rand, ycandidates_rand[rand_select].unsqueeze(0)), 0)
            
            xcandidates_rand = torch.cat((xcandidates_rand[:rand_select], xcandidates_rand[rand_select + 1:]))
            ycandidates_rand = torch.cat((ycandidates_rand[:rand_select], ycandidates_rand[rand_select + 1:]))
            
            # Update GP model, fit and predict
            gp = SingleTaskGP(xinit_rand, yinit_rand) 
            mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
            fit_gpytorch_mll(mll)
            
            # Predict the y values for the test set and calculate errors
            rand_ypred = gp(xtest)
            rand_ypred_mean = rand_ypred.mean.detach().numpy()
            rand_pred_mean.append(rand_ypred_mean)
            
            rand_ymae = mean_absolute_error(ytest, rand_ypred_mean)
            # print('mean absolute error: ', rand_ymae)
            rand_pred_mae.append(rand_ymae)
            
            rand_ystd = gp(xtest).stddev
            ystd_ = rand_ystd.detach().numpy()
            rand_pred_std.append(ystd_)
        random_mae_seeds.append(rand_pred_mae)
    return random_mae_seeds


In [21]:
random_runs_second_run = random_runs()

100%|██████████| 25/25 [02:25<00:00,  5.81s/it]


### QBC FUNCTION

In [ ]:

def qbc_runs() -> list:
    gp_commit_lst = []
    committee = [
        RandomForestRegressor(),
        SVR(),
        DecisionTreeRegressor()
    ]

    comit_pred_mae = []
    commit_seeds = []

    for i in tqdm(seeds):
        
        xcandidates_comit = xcandidates_original.clone()
        ycandidates_comit = ycandidates_original.clone()
        xinit_comit, yinit_comit, xcandidates_comit, ycandidates_comit = random_initial_data(xcandidates_comit, ycandidates_comit, 0.05, seed=i)
        gp_commit_lst = []
        
        
        gp = SingleTaskGP(xinit_comit, yinit_comit)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_mll(mll)
        posterior = gp(xtest)
        ypred = posterior.mean.detach().numpy()
        
        comitt_ymae = mean_absolute_error(ytest, ypred)
        gp_commit_lst.append(comitt_ymae)
        for inner_i in tqdm(range(len(xcandidates_comit))):
            if not len(xcandidates_comit):
                print('empty')
                break
            # print(f'this is {i}')
            for model in committee:
                model.fit(xinit_comit, yinit_comit)

            predictions = np.array([model.predict(xcandidates_comit) for model in committee])

        
            disagreement_scores = np.var(predictions, axis=0)

            N = 1  
            top_N_indices = np.argsort(disagreement_scores)[-N:]

            X_to_query = xcandidates_comit[top_N_indices]
            
            ylabel = ycandidates_comit[top_N_indices]
            
            xinit_comit = torch.cat((xinit_comit, X_to_query), 0)
            yinit_comit = torch.cat((yinit_comit, ylabel), 0)
            
            xcandidates_comit = torch.cat((xcandidates_comit[:int(top_N_indices)], xcandidates_comit[int(top_N_indices) + 1:]))
            # print(f'len of x candidates: {len(xcandidates_comit)}')
            ycandidates_comit = torch.cat((ycandidates_comit[:int(top_N_indices)], ycandidates_comit[int(top_N_indices) + 1:]))
            
            
            gp = SingleTaskGP(xinit_comit, yinit_comit)
            mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
            fit_gpytorch_mll(mll)
            posterior = gp(xtest)
            ypred = posterior.mean.detach().numpy()
                
            comitt_ymae = mean_absolute_error(ytest, ypred)
            gp_commit_lst.append(comitt_ymae)
            # print(f'length of gp_commit_lst: {len(gp_commit_lst)}')
        commit_seeds.append(gp_commit_lst)
        
        
    return commit_seeds    


In [33]:
qbc_runs = qbc_runs()

100%|██████████| 25/25 [02:56<00:00,  7.05s/it]


In [ ]:
# np.save('AM_qbc_runs.npy', np.array(qbc_runs))

### Uncertainty Sampling Function

In [ ]:

def uncertainty_runs() -> list:
    torch.manual_seed(13)
    
    uncr_xmax_candidates = []
    uncr_pred_mae = []
    uncr_pred_std = []
    uncr_pred_mean = []
    unc_rand_mae_seeds = []

    for i in seeds:        
        xcandidates_uncr = xcandidates_original.clone()
        ycandidates_uncr = ycandidates_original.clone()
        xinit_uncr, yinit_uncr, xcandidates_uncr, ycandidates_uncr = random_initial_data(xcandidates_uncr, ycandidates_uncr, 0.05, seed=i)
        
        uncr_xmax_candidates = []
        uncr_pred_mae = []
        uncr_pred_std = []
        uncr_pred_mean = []

        gp = SingleTaskGP(xinit_uncr, yinit_uncr)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_mll(mll)

        uncr_ypred = gp(xtest)
        uncr_ypred_mean = uncr_ypred.mean.detach().numpy()
        uncr_pred_mean.append(uncr_ypred_mean)
        
        uncr_ymae = mean_absolute_error(ytest, uncr_ypred_mean)
        uncr_pred_mae.append(uncr_ymae)
        
        uncr_ystd = gp(xtest).stddev.detach().numpy()
        uncr_pred_std.append(uncr_ystd)

        # Active learning loop (25 iterations)
        for inner_i in tqdm(range(len(xcandidates_uncr))):
            if not len(xcandidates_uncr):
                print('empty')
                break
            posterior_candidates = gp(xcandidates_uncr)
            uncertainties = posterior_candidates.stddev.detach().numpy() 
            max_uncertainty_idx = uncertainties.argmax()

            xinit_uncr = torch.cat((xinit_uncr, xcandidates_uncr[max_uncertainty_idx].unsqueeze(0)), 0)
            yinit_uncr = torch.cat((yinit_uncr, ycandidates_uncr[max_uncertainty_idx].unsqueeze(0)), 0)
            
            xcandidates_uncr = torch.cat((xcandidates_uncr[:max_uncertainty_idx], xcandidates_uncr[max_uncertainty_idx + 1:]))
            ycandidates_uncr = torch.cat((ycandidates_uncr[:max_uncertainty_idx], ycandidates_uncr[max_uncertainty_idx + 1:]))

            # Retrain the GP model on the updated training set
            gp = SingleTaskGP(xinit_uncr, yinit_uncr)
            mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
            fit_gpytorch_mll(mll)

            # Predict the y values for the test set
            uncr_ypred = gp(xtest)
            uncr_ypred_mean = uncr_ypred.mean.detach().numpy()
            uncr_pred_mean.append(uncr_ypred_mean)

            # Calculate the mean absolute error (MAE) for the test set
            uncr_ymae = mean_absolute_error(ytest, uncr_ypred_mean)
            # print(f'Iteration {i}: mean absolute error = {uncr_ymae}')
            uncr_pred_mae.append(uncr_ymae)

            uncr_ystd = gp(xtest).stddev.detach().numpy()
            uncr_pred_std.append(uncr_ystd)

        unc_rand_mae_seeds.append(uncr_pred_mae)
    return unc_rand_mae_seeds

In [295]:
uncertainty_runs = uncertainty_runs()

100%|██████████| 67/67 [00:06<00:00, 10.77it/s]


In [296]:
np.save('AM_uncertainty_runs.npy', np.array(uncertainty_runs))